In [1]:
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm


In [2]:
train_path = '../../user_data/dataset/'
test_path = '../../user_data/dataset/'
header = 'underexpose'
now_phase = 9

In [3]:
item_count_dict = {}
user_phase_dict = {}

whole_click = pd.DataFrame() 
for c in range(now_phase + 1):  
    #print('phase:', c)  
    click_train = pd.read_csv(train_path + header + '_train_click_{}_time.csv'.format(c))  
    click_test = pd.read_csv(test_path + header + '_test_click_{}_time.csv'.format(c))  
    click_query = pd.read_csv(test_path + header + '_test_qtime_{}_time.csv'.format(c)) 

    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  
    df_item_count = whole_click.groupby(['item_id'],as_index=False)['user_id'].agg({'count':'count'}) 
    
    for i, row in df_item_count.iterrows():
        item_count_dict.setdefault(row['item_id'],list(np.zeros(now_phase+1)))
        item_count_dict[row['item_id']][c] = row['count']   
        
    
    for i, row in click_query.iterrows():
        user_phase_dict[row['user_id']] = c

In [4]:
item_count_df = pd.DataFrame({'item_id': list(item_count_dict.keys()), 'count': list(item_count_dict.values())})

In [5]:
item_count_list = []

for i,row in item_count_df.iterrows():
    for j in range(now_phase + 1):
        item_count_list.append([row['item_id'],j,row['count'][j]])

item_count_df = pd.DataFrame(item_count_list, columns=['item_id', 'phrase', 'count'])

In [6]:
# 与上下阶段的差

item_count_df_list = []
for i,x in tqdm(item_count_df.groupby('item_id')):
    x['diff_from_last'] = x['count'].diff(1)
    x['diff_from_next'] = x['count'].diff(-1)
    item_count_df_list.append(x)

item_count_df = pd.concat(item_count_df_list)


# 最大值与当前phase的差

item_count_df['max'] = item_count_df.groupby('item_id')['count'].transform('max')
item_count_df['diff_from_max'] = item_count_df['max'] - item_count_df['count']
item_count_df = item_count_df.drop(columns = 'max')

  0%|          | 0/117720 [00:00<?, ?it/s]/home/wangdi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/wangdi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 117720/117720 [03:24<00:00, 575.71it/s]


In [7]:
file_name = 'recall_0531_addsim_addAA_RA_additemtime'

df = pd.read_csv('../dataset/new_recall/' + file_name + '.csv')

In [8]:
df.shape

(17383059, 26)

In [9]:
df['phrase'] = df['user_id'].apply(lambda x:user_phase_dict[x])

In [10]:
df = pd.merge(left = df,
             right = item_count_df,
             how = 'left',
             on = ['item_id','phrase'])

In [11]:
df.to_csv('../../user_data/dataset/new_recall/' + file_name + '_addcount.csv',index=False)

In [12]:
df.shape

(17383059, 31)

In [13]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  ...  item_count_12h  item_count_6h  \
0     0.0000   490.3025          0  ...             3.0            NaN   
1     0.0000   472.3559          0  ...             4.0            NaN   
2     0.2152   487.1326          4  ...             6.0            3.0   
3     0.0000   472.6520          0  ...             2.0            1.0   
4     0.0000   485.4152          0  ...             3.0            1.0   

   item_count_4h  item_count_2h  item_count_1h  phrase  count  diff_from_last  \
0            1.0            NaN            NaN       0   14.0             NaN   
1            NaN            NaN            NaN       0    7.0             NaN   
2            1.0            NaN            NaN       0   17.0             NaN   
3            NaN            NaN            NaN       0    9.0             NaN   
4            NaN            NaN            NaN       0    7.0             NaN   

   diff_from_next  diff_from_max  
0           -14.0           94.0  
1            -9.0           21.0  
2           -18.0          125.0  
3            -5.0           65.0  
4            -6.0           22.0  

[5 rows x 31 columns]